In [ ]:
!pip install fastapi uvicorn diffusers transformers peft accelerate torch safetensors

  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from diffusers import StableDiffusionPipeline

# This will automatically download all required components into the cache
pipeline = StableDiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0")

# Save it locally for custom loading
pipeline.save_pretrained("/content/stable-diffusion-xl-base-1.0")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/575 [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

...

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vae_1_0/diffusion_pytorch_model.safetens(…):   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [ ]:
from fastapi import FastAPI, HTTPException, UploadFile, Form
from fastapi.responses import FileResponse
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware
from diffusers import StableDiffusionPipeline, UNet2DConditionModel, AutoencoderKL, DDIMScheduler
from transformers import CLIPTextModel, CLIPTokenizer
import torch
import os
# from peft import PeftMode
app = FastAPI()

# Allow cross-origin requests for frontend communication
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # For dev; restrict in production
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Load base model components
# base_path = "/content/stable-diffusion-v1-5"
base_path ="runwayml/stable-diffusion-v1-5"
vae = AutoencoderKL.from_pretrained(base_path, subfolder="vae", torch_dtype=torch.float16)
text_encoder = CLIPTextModel.from_pretrained(base_path, subfolder="text_encoder", torch_dtype=torch.float16)
tokenizer = CLIPTokenizer.from_pretrained(base_path, subfolder="tokenizer")
unet = UNet2DConditionModel.from_pretrained(base_path, subfolder="unet", torch_dtype=torch.float16)
scheduler = DDIMScheduler.from_pretrained(base_path, subfolder="scheduler")

pipe = StableDiffusionPipeline(
    vae=vae,
    text_encoder=text_encoder,
    tokenizer=tokenizer,
    unet=unet,
    scheduler=scheduler,
    safety_checker=None,
    feature_extractor=None,
    requires_safety_checker=False
).to("cuda")  # Ensure CUDA is available, else use "cpu"
# pipe.unet = torch.compile(pipe.unet, mode="reduce-overhead")
# Optional compilation
# pipe.unet = torch.compile(pipe.unet, mode="reduce-overhead")



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

prompt = "Extreme close-up of a single tiger eye, direct frontal view. Detailed iris and pupil. Sharp focus on eye texture and color. Natural lighting to capture authentic eye shine and depth. The word 'FLUX' is painted over it in big, white brush strokes with visible texture."

print("Checkpoint 0")

try:
    result = pipe(
        prompt=prompt,
        height=512,
        width=512,
        num_inference_steps=20,
        guidance_scale=7.5
    )
    print("Checkpoint1")
    image = result.images[0]
    image.save("output_lora.png")
    print("✅ Saved as output_lora.png")
except Exception as e:
    print("❌ Failed to generate image:", str(e))


Checkpoint 0


  0%|          | 0/20 [00:00<?, ?it/s]

Checkpoint1
✅ Saved as output_lora.png


In [ ]:
# LoRA loading endpoint
@app.post("/load-lora/")
async def load_lora(
    lora_path: str = Form(...)
):
    try:
        pipe.load_lora_weights(lora_path)
        return {"message": f"LoRA loaded from {lora_path}"}
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Failed to load LoRA: {str(e)}")


# Prompt input model
class PromptRequest(BaseModel):
    prompt: str
    height: int = 512
    width: int = 512
    num_inference_steps: int = 20
    guidance_scale: float = 7.5

# Image generation endpoint
# @app.post("/generate-image/")
# async def generate_image(request: PromptRequest):
#     image_path = "output_lora.png"
#     try:
#         image = pipe(
#             prompt=request.prompt,
#             height=request.height,
#             width=request.width,
#             num_inference_steps=request.num_inference_steps,
#             guidance_scale=request.guidance_scale
#         ).images[0]

#         image.save(image_path)
#         return FileResponse(image_path, media_type="image/png")
#     except Exception as e:
#         raise HTTPException(status_code=500, detail=str(e))
#     finally:
#         if os.path.exists(image_path):
#             os.remove(image_path)
import traceback

@app.post("/generate-image/")
async def generate_image(request: PromptRequest):
    image_path = "/content/output_lora.png"
    try:
        print("⚙️ Generating with prompt:", request.prompt)
        print("Prompt:", request.prompt)
        print("Width:", request.width, "Height:", request.height)
        print("Steps:", request.num_inference_steps, "Scale:", request.guidance_scale)
        print("Checkpoint 0")
        image = pipe(
            prompt=request.prompt,
            height=request.height,
            width=request.width,
            num_inference_steps=request.num_inference_steps,
            guidance_scale=request.guidance_scale,
        ).images[0]
        print("⏳ Image generation complete.")
        print("Image list:", image.images)
        print("Checkpoint1")
        image.show()  # Display in notebook
        image.save(image_path)
        return FileResponse(image_path, media_type="image/png")
    except Exception as e:
        print("❌ Exception occurred:")
        traceback.print_exc()  # ← VERY IMPORTANT
        raise HTTPException(status_code=500, detail=str(e))
    finally:
        if os.path.exists(image_path):
            os.remove(image_path)



In [ ]:
# LoRA loading endpoint
@app.post("/load-lora/")
async def load_lora(
    lora_path: str = Form(...)
):
    try:
        pipe.load_lora_weights(lora_path)
        return {"message": f"LoRA loaded from {lora_path}"}
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Failed to load LoRA: {str(e)}")


# Prompt input model
class PromptRequest(BaseModel):
    prompt: str
    height: int = 512
    width: int = 512
    num_inference_steps: int = 20
    guidance_scale: float = 7.5

import traceback
import asyncio
from concurrent.futures import ThreadPoolExecutor
from fastapi.responses import FileResponse
from io import BytesIO
from fastapi.responses import StreamingResponse
executor = ThreadPoolExecutor()

def generate_image_sync(prompt, height, width, steps, scale):
    return pipe(
        prompt=prompt,
        height=height,
        width=width,
        num_inference_steps=steps,
        guidance_scale=scale
    ).images[0]


@app.post("/generate-image/")
async def generate(request: PromptRequest):
  try:
      print("⚙️ Generating with prompt:", request.prompt)
      print("Prompt:", request.prompt)
      print("Width:", request.width, "Height:", request.height)
      print("Steps:", request.num_inference_steps, "Scale:", request.guidance_scale)
      print("Checkpoint 0")
      image = await asyncio.get_event_loop().run_in_executor(
          executor,
          generate_image_sync,
          request.prompt,
          request.height,
          request.width,
          request.num_inference_steps,
          request.guidance_scale
      )
      print("Checkpoint 1")
      # image_path = "/content/output_lora.png"
      # image.save(image_path)
      # return FileResponse(image_path, media_type="image/png", filename="output_lora.png")
      img_bytes = BytesIO()
      image.save(img_bytes, format="PNG")
      img_bytes.seek(0)
      print("Bytes length:", len(img_bytes.getvalue()))
      return StreamingResponse(
          img_bytes,
          media_type="image/png",
          headers={
              "Content-Disposition": 'inline; filename="output.png"',
              "Content-Type": "image/png"
          }
      )
  except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

In [ ]:
# from fastapi.responses import FileResponse
# import uuid
# import os

# def generate_image_sync(prompt, height, width, steps, scale):
#     image = pipe(
#         prompt=prompt,
#         height=height,
#         width=width,
#         num_inference_steps=steps,
#         guidance_scale=scale
#     ).images[0]
#     return image

# @app.post("/generate-image/")
# async def generate(request: PromptRequest):
#     print("⚙️ Generating with prompt:", request.prompt)
#     print("Prompt:", request.prompt)
#     print("Width:", request.width, "Height:", request.height)
#     print("Steps:", request.num_inference_steps, "Scale:", request.guidance_scale)
#     print("Checkpoint 0")

#     image = await asyncio.get_event_loop().run_in_executor(
#         executor,
#         generate_image_sync,
#         request.prompt,
#         request.height,
#         request.width,
#         request.num_inference_steps,
#         request.guidance_scale
#     )

#     print("Checkpoint 1")

#     # ✅ Use a unique filename to avoid browser caching issues
#     filename = f"output_{uuid.uuid4().hex}.png"
#     image_path = f"/content/{filename}"
#     image.save(image_path, format="PNG")

#     # ✅ Confirm the file is saved and readable
#     if not os.path.exists(image_path):
#         raise RuntimeError(f"Image not saved at {image_path}")

#     print(f"✅ Saved image at: {image_path}")
#     return FileResponse(path=image_path, media_type="image/png", filename=filename)


In [ ]:
!pip install pyngrok

In [ ]:
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
import torch
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
import uvicorn
from pyngrok import ngrok
import nest_asyncio
!ngrok authtoken 2xNBjsJZTEMSyZIjuNauqUwXAxK_3YEAxPqFauzLBSHA8EJvn
# Allow async in Colab
nest_asyncio.apply()

# Start ngrok tunnel
public_url = ngrok.c3onnect(8000)
print("Public URL:", public_url)

# Run FastAPI server
uvicorn.run(app, host="0.0.0.0", port=8000)


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Public URL: NgrokTunnel: "https://ab24-34-87-30-222.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [453]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


⚙️ Generating with prompt: Extreme close-up of a single tiger eye, direct frontal view. Detailed iris and pupil. Sharp focus on eye texture and color. Natural lighting to capture authentic eye shine and depth. The word FLUX is painted over it in big, white brush strokes with visible texture.
Prompt: Extreme close-up of a single tiger eye, direct frontal view. Detailed iris and pupil. Sharp focus on eye texture and color. Natural lighting to capture authentic eye shine and depth. The word FLUX is painted over it in big, white brush strokes with visible texture.
Width: 512 Height: 512
Steps: 20 Scale: 7.5
Checkpoint 0


  0%|          | 0/20 [00:00<?, ?it/s]

Checkpoint 1
Bytes length: 456910
INFO:     35.185.102.34:0 - "POST /generate-image/ HTTP/1.1" 200 OK
INFO:     2409:40e4:1157:529d:c007:e880:ac39:5c4f:0 - "OPTIONS /generate-image/ HTTP/1.1" 200 OK
⚙️ Generating with prompt: Extreme close-up of a single tiger eye, direct frontal view. Detailed iris and pupil. Sharp focus on eye texture and color. Natural lighting to capture authentic eye shine and depth. The word "FLUX" is painted over it in big, white brush strokes with visible texture.
Prompt: Extreme close-up of a single tiger eye, direct frontal view. Detailed iris and pupil. Sharp focus on eye texture and color. Natural lighting to capture authentic eye shine and depth. The word "FLUX" is painted over it in big, white brush strokes with visible texture.
Width: 512 Height: 512
Steps: 20 Scale: 7.5
Checkpoint 0


...

The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['texture .']


⚙️ Generating with prompt: Extreme close-up of a fresh sunflower center showing intricate spiral seed pattern. Sharp focus on individual florets with geometric precision. Morning dew droplets catch golden hour light creating micro-reflections. Rich browns and yellows dominate the color palette. The word "BLOOM" is hand-lettered in flowing white paint with thick, impasto brush technique showing dimensional texture.
Prompt: Extreme close-up of a fresh sunflower center showing intricate spiral seed pattern. Sharp focus on individual florets with geometric precision. Morning dew droplets catch golden hour light creating micro-reflections. Rich browns and yellows dominate the color palette. The word "BLOOM" is hand-lettered in flowing white paint with thick, impasto brush technique showing dimensional texture.
Width: 512 Height: 512
Steps: 20 Scale: 7.5
Checkpoint 0


  0%|          | 0/20 [00:00<?, ?it/s]

Checkpoint 1
Bytes length: 556954
INFO:     2401:4900:8820:e735:441e:6f77:35fa:15df:0 - "POST /generate-image/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [453]
